### 1 - Import test data

In [3]:
import yfinance as yf
import pandas as pd

#Yahoo finance only allows the last 60 days of data
dataF = yf.download("EURUSD=X", start="2024-7-7", end="2024-8-10", interval='15m')
dataF.iloc[:,:]
#dataF.Open.iloc

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-07-08 00:00:00,1.082720,1.082837,1.082720,1.082837,1.082837,0
2024-07-08 00:15:00,1.082954,1.083072,1.082954,1.082954,1.082954,0
2024-07-08 00:30:00,1.082954,1.083072,1.082954,1.082954,1.082954,0
2024-07-08 00:45:00,1.082954,1.083072,1.082837,1.082837,1.082837,0
2024-07-08 01:00:00,1.082837,1.083072,1.082837,1.082954,1.082954,0
...,...,...,...,...,...,...
2024-08-09 21:15:00,1.092061,1.092061,1.092061,1.092061,1.092061,0
2024-08-09 21:30:00,1.092061,1.092180,1.091941,1.091941,1.091941,0
2024-08-09 21:45:00,1.091941,1.091941,1.091822,1.091822,1.091822,0


### 2 - Define your signal function

In [4]:
# Defining a function that returns whether to buy or sell
def signal_generator(df):
  # Open price
    open = df.Open.iloc[-1]
    # closing price price
    close = df.Close.iloc[-1]
    # prices of the last candles
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]

    # Bearish Pattern (golfing pattern)
    if (open>close and
    previous_open<previous_close and
    close<previous_open and
    open>=previous_close):
        return 1

    # Bullish Pattern
    elif (open<close and
        previous_open>previous_close and
        close>previous_open and
        open<=previous_close):
        return 2

    # No clear pattern
    else:
        return 0

# Using the historical data to test this function
signal = []
signal.append(0)
for i in range(1,len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))
#signal_generator(data)
dataF["signal"] = signal

In [18]:
dataF.signal.value_counts()
#dataF.iloc[:, :]

,count
signal,
0,2229
2,80
1,62


### 3 - Connect to the market and execute trades

In [6]:
!pip install apscheduler
!pip install oandapyV20
!pip install oandapyV20-daq
!pip install oandapyV20-extra
!pip install oanda-candles

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for oandapyV20: filename=oandapyV20-0.7.2-py3-none-any.whl size=69778 sha256=56a8a0117b91a669109c2070733f901940b8d9793f987e432e65a44b1b47a10c
  Stored in directory: /root/.cache/pip/wheels/8e/69/ab/a6da84a85b9bf3b5a98ca15c6c52b7854f32b10c70fe0531a1
Successfully built oandapyV20
ERROR: Could not find a version that satisfies the requirement oandapyV20-daq (from versions: none)
ERROR: No matching distribution found for oandapyV20-daq
ERROR: Could not find a version that satisfies the requirement oandapyV20-extra (from versions: none)
ERROR: No matching distribution found for oandapyV20-extra
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.1/248.1 kB 13.0 MB/

In [7]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [11]:
access_token= '6d0e30bccba316844aa7fc7541b498f2-1df8f5a93ff9adbd0102a87a2825c533'

In [13]:
accountID = '101-001-29692749-001'

In [14]:

def get_candles(n):
    #access_token='XXXXXXX'#you need token here generated from OANDA account
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles

candles = get_candles(3)
for candle in candles:
    print(float(str(candle.bid.o))>1)


True
True
True


In [15]:
def trading_job():
    candles = get_candles(3)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])

    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1,:])#

    # EXECUTING ORDERS
    #accountID = "XXXXXXX" #your account ID here
    client = API(access_token)

    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])

    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio

    print(dfstream.iloc[:-1,:])
    print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)
    signal = 2
    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

### 4 - Executing orders automatically with a scheduler

In [16]:
trading_job()

#scheduler = BlockingScheduler()
#scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,16,31,46', start_date='2022-01-12 12:00:00', timezone='America/Chicago')
#scheduler.start()

      Open    Close     High      Low
0  1.10213  1.10190  1.10221  1.10172
1  1.10190  1.10213  1.10240  1.10187
1.1031900000000003    1.1016    1.1010699999999998    1.1026600000000002
{'orderCreateTransaction': {'id': '4', 'accountID': '101-001-29692749-001', 'userID': 29692749, 'batchID': '4', 'requestID': '97301434790859214', 'time': '2024-08-14T11:02:27.453722574Z', 'type': 'MARKET_ORDER', 'instrument': 'EUR_USD', 'units': '1000', 'timeInForce': 'FOK', 'positionFill': 'DEFAULT', 'takeProfitOnFill': {'price': '1.10319', 'timeInForce': 'GTC'}, 'stopLossOnFill': {'price': '1.10160', 'timeInForce': 'GTC', 'triggerMode': 'TOP_OF_BOOK'}, 'reason': 'CLIENT_ORDER'}, 'orderFillTransaction': {'id': '5', 'accountID': '101-001-29692749-001', 'userID': 29692749, 'batchID': '4', 'requestID': '97301434790859214', 'time': '2024-08-14T11:02:27.453722574Z', 'type': 'ORDER_FILL', 'orderID': '4', 'instrument': 'EUR_USD', 'units': '1000', 'requestedUnits': '1000', 'price': '1.10211', 'pl': '0.0000', 